# OTX API DEMO (MMPretrain Example)

## Customization Training API

Select a framework & import adapter modules.

We'll choose MMpretrain here.

"""
Environment:
- mmpretrain-1.0.0rc8
- mmcv-2.0.1
- mmengine-0.7.4
- mmdeploy-1.2.0

"""

## Prepare Dataset & DataLoader
1. Prepare a dataset and enter path into Dataset

    - Convert to OTX's DatasetEntity and Label Schema by leveraging Datumaro's features through paths (path -> Datumaro -> OTX DatasetEntity & LabelSchema)

In [1]:
from otx.v2.adapters.torch.mmengine.mmpretrain import Dataset
dataset = Dataset(
    train_data_roots="/home/harimkan/workspace/repo/otx-fork-3/tests/assets/classification_dataset_class_incremental",
    val_data_roots="/home/harimkan/workspace/repo/otx-fork-3/tests/assets/classification_dataset_class_incremental",
    test_data_roots="/home/harimkan/workspace/repo/otx-fork-3/tests/assets/classification_dataset_class_incremental",
)

/home/harimkan/workspace/repo/otx-fork-3/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/harimkan/workspace/repo/otx-fork-3/venv/lib/python3.9/site-packages/openvino/pyopenvino/__init__.py:10: FutureWarning: The module is private and following namespace `pyopenvino` will be removed in the future
  warnings.warn(message="The module is private and following namespace " "`pyopenvino` will be removed in the future", category=FutureWarning)


2-1. Build Torch Dataset from MMCV config (filepath or dict) -> torch.utils.data.Dataset

    - User can build a dataset from a config file or dictionary.

In [2]:
train_dataloader = dataset.train_dataloader()
print(f"Dataset type: {type(train_dataloader)}")
print(f"Length of DataLoader: {len(train_dataloader)}")
print(f"Dataset size: {len(train_dataloader.dataset)}")
print(f"Number of classes: {dataset.num_classes}")

[*] Detected dataset format: imagenet
[*] Detected task type: CLASSIFICATION
2023-07-14 09:25:43,324 | INFO : Try to create a 0 size memory pool.
Dataset type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 32
Dataset size: 32
Number of classes: 3


In [3]:
# Customize batch_size
train_dataloader = dataset.train_dataloader(batch_size=4)
print(f"DataLoader type: {type(train_dataloader)}")
print(f"Length of DataLoader: {len(train_dataloader)}")
print(f"Dataset size: {len(train_dataloader.dataset)}")

DataLoader type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 8
Dataset size: 32


In [4]:
val_dataloader = dataset.val_dataloader()
print(f"DataLoader type: {type(val_dataloader)}")
print(f"Length of DataLoader: {len(val_dataloader)}")
print(f"Dataset size: {len(val_dataloader.dataset)}")

DataLoader type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 32
Dataset size: 32


In [5]:
test_dataloader = dataset.test_dataloader()
print(f"DataLoader type: {type(test_dataloader)}")
print(f"Length of DataLoader: {len(test_dataloader)}")
print(f"Dataset size: {len(test_dataloader.dataset)}")

DataLoader type: <class 'torch.utils.data.dataloader.DataLoader'>
Length of DataLoader: 32
Dataset size: 32


## Prepare Model
### Model provided by OTX

In [6]:
## OTX Custom Model
from otx.v2.adapters.torch.mmengine.mmpretrain import get_model
otx_model = get_model(
    config="../configs/classification/otx_mmpretrain_efficientnet_b0.yaml",
    num_classes=dataset.num_classes
)
print(f"Model type: {type(otx_model)}")

2023-07-14 09:25:54,524 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
2023-07-14 09:25:54,549 | INFO : 'in_channels' config in model.head is updated from -1 to 1280
2023-07-14 09:25:54,614 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
Model type: <class 'otx.v2.adapters.torch.mmengine.mmpretrain.modules.models.classifiers.sam_classifier.SAMImageClassifier'>


### Model provided by mmpretrain

In [7]:
# mmpretrain's pre-defined model
from mmpretrain import get_model
mmpretrain_model = get_model("resnet18_8xb32_in1k")
print(f"Model type: {type(mmpretrain_model)}")

Model type: <class 'mmpretrain.models.classifiers.image.ImageClassifier'>


## Training

Users can use each framework's training provided by OTX. (Engine)

- The engine requires the necessary models and DataLoaders for each framework.

In [8]:
from otx.v2.adapters.torch.mmengine.mmpretrain.engine import MMPTEngine

# OTX Model Training
engine = MMPTEngine(work_dir="/tmp/otx-test",)

# Training without validation
trained_model, ckpt_path = engine.train(
    model=otx_model,
    train_dataloader=train_dataloader,
    max_epochs=3,
)

print(ckpt_path)

07/14 09:26:02 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.13 (main, Aug 25 2022, 23:26:10) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 1275199889
    GPU 0,1: NVIDIA GeForce RTX 3090
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.7, V11.7.64
    GCC: gcc (Ubuntu 9.5.0-1ubuntu1~22.04) 9.5.0
    PyTorch: 1.13.1+cu117
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.7
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=

In [9]:
val_score = engine.validate(val_dataloader=val_dataloader)
print(f"Val Metric: {val_score}")

test_score = engine.test(test_dataloader=test_dataloader)
print(f"Test Metric: {test_score}")

07/14 09:26:07 - mmengine - INFO - Epoch(val) [3][32/32]    accuracy/top1: 68.7500  data_time: 0.0003  time: 0.0062


07/14 09:26:08 - mmengine - INFO - The best checkpoint with 68.7500 accuracy/top1 at 3 epoch is saved to best_accuracy_top1_epoch_3.pth.
Val Metric: {'accuracy/top1': 68.75}
07/14 09:26:08 - mmengine - INFO - Epoch(test) [32/32]    accuracy/top1: 68.7500  data_time: 0.0003  time: 0.0061
Test Metric: {'accuracy/top1': 68.75}


In [10]:
# OR
# Training with validation
trained_model, ckpt_path = engine.train(
    model=otx_model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    max_epochs=3,
)

print(ckpt_path)

07/14 09:26:10 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.13 (main, Aug 25 2022, 23:26:10) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 1486445472
    GPU 0,1: NVIDIA GeForce RTX 3090
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.7, V11.7.64
    GCC: gcc (Ubuntu 9.5.0-1ubuntu1~22.04) 9.5.0
    PyTorch: 1.13.1+cu117
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.7
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=

07/14 09:26:11 - mmengine - WARNING - init_weights of SAMImageClassifier has been called more than once.
07/14 09:26:11 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
07/14 09:26:11 - mmengine - INFO - Checkpoints will be saved to /tmp/otx-test.
07/14 09:26:11 - mmengine - INFO - Exp name: 20230714_092610
07/14 09:26:11 - mmengine - INFO - Epoch(train) [1][8/8]  lr: 1.0000e-02  eta: 0:00:00  time: 0.0261  data_time: 0.0011  memory: 413  loss: 1.0072
07/14 09:26:11 - mmengine - INFO - Saving checkpoint at 1 epochs
07/14 09:26:11 - mmengine - WARNING - `save_param_scheduler` is True but `self.param_schedulers` is None, so skip saving parameter schedulers
07/14 09:26:11 - mmengine - INFO - Epoch(val) [1][32/32]    accuracy/top1: 84.3750  data_time: 0.0003  time: 0.0061
07/14 09:26:11 - mmengine - INFO - The best checkpoint with 84.3750 accuracy/top1 at 1 epoch is saved to best_a

In [11]:
val_score = engine.validate()
print(f"Val Metric: {val_score}")

test_score = engine.test(test_dataloader=test_dataloader)
print(f"Test Metric: {test_score}")

07/14 09:26:18 - mmengine - INFO - Epoch(val) [3][32/32]    accuracy/top1: 50.0000  data_time: 0.0003  time: 0.0061


Val Metric: {'accuracy/top1': 50.0}
07/14 09:26:18 - mmengine - INFO - Epoch(test) [32/32]    accuracy/top1: 50.0000  data_time: 0.0003  time: 0.0061
Test Metric: {'accuracy/top1': 50.0}


## Predict

In [12]:
sample = "/home/harimkan/workspace/repo/otx-fork-3/tests/assets/classification_dataset_class_incremental/2/22.jpg"

predict_output = engine.predict(
    model=trained_model,
    img=sample
)

print(predict_output)

[{'pred_scores': array([9.999937e-01, 2.390463e-17, 6.342143e-06], dtype=float32), 'pred_label': 0, 'pred_score': 0.9999936819076538}]


## Export (with mmdeploy)

In [13]:
export_output = engine.export(
    model=trained_model
)

print(export_output)

07/14 09:26:26 - mmengine - WARNING - Failed to search registry with scope "mmengine" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmengine" is a correct scope, or whether the registry is initialized.
07/14 09:26:26 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmpretrain" is a correct scope, or whether the registry is initialized.
07/14 09:26:26 - mmengine - WARNING - Failed to search registry with scope "mmpretrain" in the "mmpretrain_tasks" registry tree. As a workaround, the current "mmpretrain_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failu

/home/harimkan/workspace/repo/otx-fork-3/venv/lib/python3.9/site-packages/mmpretrain/utils/setup_env.py:34: UserWarning: The current default scope "mmengine" is not "mmpretrain", `register_all_modules` will force the current default scope to be "mmpretrain". If this is not expected, please set `init_default_scope=False`.
  warnings.warn(


07/14 09:26:28 - mmengine - INFO - Args for Model Optimizer: mo --input_model="/tmp/otx-test/openvino.onnx" --output_dir="/tmp/otx-test/" --output="output" --input="input" --input_shape="[1, 3, 224, 224]" --mean_values="[123.675, 116.28, 103.53]" --scale_values="[58.395, 57.12, 57.375]" 
07/14 09:26:30 - mmengine - INFO - Check for a new version of Intel(R) Distribution of OpenVINO(TM) toolkit here https://software.intel.com/content/www/us/en/develop/tools/openvino-toolkit/download.html?cid=other&source=prod&campid=ww_2023_bu_IOTG_OpenVINO-2022-3&content=upg_all&medium=organic or on https://github.com/openvinotoolkit/openvino
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.

## OTX AutoEngine (Automation Training API)
OTX provides a more convenient API called AutoEngine.

- It's more convenient for users to use Engine, which provides auto-configuration and the features provided by OTX without having to choose a framework.
- Prepare Dataset & DataLoader + Prepare Model + OTX Recipes + Training + ETC.
- This will make all of the above steps happen automatically. (Auto: Model Selection & build, Dataset Configuration, Training, etc..)

In [14]:
from otx.v2.api.core.engine import AutoEngine

output_dir = "/tmp/OTX-API-test"
data_roots = "../../../../tests/assets/classification_dataset_class_incremental"
default_config_path = "../configs/classification/otx_mmpretrain_efficientnet_b0.yaml"

engine = AutoEngine(
    work_dir=output_dir,
    train_data_roots=data_roots,
    config=default_config_path,
)

# Customization training
trained_model, ckpt_path = engine.train(batch_size=2, max_epochs=2)
print(ckpt_path)

2023-07-14 09:27:23,309 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
2023-07-14 09:27:23,330 | INFO : 'in_channels' config in model.head is updated from -1 to 1280
2023-07-14 09:27:23,396 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip


07/14 09:27:23 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.13 (main, Aug 25 2022, 23:26:10) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 870339434
    GPU 0,1: NVIDIA GeForce RTX 3090
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.7, V11.7.64
    GCC: gcc (Ubuntu 9.5.0-1ubuntu1~22.04) 9.5.0
    PyTorch: 1.13.1+cu117
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.7
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=s

In [15]:
# With Validation
engine = AutoEngine(
    work_dir=output_dir,
    train_data_roots=data_roots,
    val_data_roots=data_roots,
    config=default_config_path,
)

# Customization training
trained_model, ckpt_path = engine.train(batch_size=2, max_epochs=2)
print(ckpt_path)

2023-07-14 09:27:29,973 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip
2023-07-14 09:27:29,996 | INFO : 'in_channels' config in model.head is updated from -1 to 1280
2023-07-14 09:27:30,065 | INFO : init weight - https://github.com/osmr/imgclsmob/releases/download/v0.0.364/efficientnet_b0-0752-0e386130.pth.zip


07/14 09:27:30 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.13 (main, Aug 25 2022, 23:26:10) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 58948292
    GPU 0,1: NVIDIA GeForce RTX 3090
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.7, V11.7.64
    GCC: gcc (Ubuntu 9.5.0-1ubuntu1~22.04) 9.5.0
    PyTorch: 1.13.1+cu117
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.7
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm